In [23]:
from IPython.core.display import display, HTML

In [24]:
print("If you want to follow along, checkout this pull request:")
display(HTML("""<a href="https://github.com/yeatmanlab/pyAFQ/pull/289">https://github.com/yeatmanlab/pyAFQ/pull/289</a>"""))
print("To use checkout pull request, run the following commands in your pyAFQ folder:")
print("    git checkout -b 36000-plotly master")
print("    git pull https://github.com/36000/pyAFQ.git plotly")
print("\nThis jupyter notebook script is saved here: ")
display(HTML("""<a href="https://github.com/36000/pyAFQ-scripts/blob/master/plotly/Plotly-demo.ipynb">https://github.com/36000/pyAFQ-scripts/blob/master/plotly/Plotly-demo.ipynb</a>"""))

If you want to follow along, checkout this pull request:


To use checkout pull request, run the following commands in your pyAFQ folder:
    git checkout -b 36000-plotly master
    git pull https://github.com/36000/pyAFQ.git plotly

This jupyter notebook script is saved here: 


In [6]:
# The next few cells pertain to setting up an afq object using prek data from s3
# If you have access to our amazon s3, you can try running this with the sample prek subject
# Otherwise, follow along with the parts of the example that do not require this data

import logging
import s3fs
import os.path as op
import os
import AFQ.api as api

In [7]:
# set up AFQ api object
def afq_prek_api(subject):    
    logging.basicConfig(level=logging.INFO)
    log = logging.getLogger(__name__)  

    fs = s3fs.S3FileSystem()
    
    sub_prefix = f"sub-"
        
    dwi_dir = f'prek-diffusion/derivatives/sub-{subject}/dtiInit_ses-pre'
    dmri_prep_dir = op.join(op.expanduser('~'), 'AFQ_data', 'prek-diffusion/dmri_prep')

    prek_fdata_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.nii.gz")
    prek_fbval_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.bvals")
    prek_fbvec_src = op.join(dwi_dir, f"sub-{subject}_ses-pre_acq-b1500_dwi_aligned_trilin.bvecs")
    
    prek_fdata = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.nii.gz"
    prek_fbval = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.bvals"
    prek_fbvec = f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/dwi/dwi_aligned_trilin.bvecs"
    
    anat_dir = f'prek-diffusion/derivatives/sub-{subject}/freesurfer/mri'

    if not op.exists(prek_fdata):
        fs.get(prek_fdata_src, prek_fdata)
        fs.get(prek_fbval_src, prek_fbval)
        fs.get(prek_fbvec_src, prek_fbvec)

        fs.get(op.join(anat_dir, f"aparc+aseg.nii.gz"), f"{dmri_prep_dir}/sub-{subject}/dtiInit_ses-pre/anat/aparc+aseg.nii.gz")

    b0_threshold = 50

    tracking_params = {"n_seeds": 2, "directions": 'det', "stop_threshold": 0.2}
    segmentation_params = {"prob_threshold": 10, "b0_threshold": b0_threshold}
    clean_params = {"clean_rounds": 1}
    
    # uncomment this to download relevant files instead of generating them
    # fs.get("prek-diffusion/derivatives_fa/afq/sub1110/sess", op.join(op.expanduser('~'), 'AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre'), recursive=True)

    myafq = api.AFQ(dmri_prep_dir,
                    dwi_folder="dwi",
                    dwi_file="dwi*", 
                    anat_folder="anat",
                    sub_prefix=sub_prefix,
                    b0_threshold=b0_threshold,
                    viz_library="plotly",

                    tracking_params=tracking_params,
                    segmentation_params=segmentation_params,
                    clean_params=clean_params)

    myafq.export_bundles()
    return myafq

In [8]:
# myafq = afq_prek_api(1110) # run on example subject

INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/templates 
INFO:dipy.data.fetcher:Dataset is already in place. If you want to fetch it again please first remove the folder /Users/john/AFQ_data/callosum_templates 
INFO:AFQ.api:Saving /Users/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-ATR_R_tractography.trk
INFO:AFQ.api:Saving /Users/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-ATR_L_tractography.trk
INFO:AFQ.api:Saving /Users/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-CGC_R_tractography.trk
INFO:AFQ.api:Saving /Users/john/AFQ_data/prek-diffusion/afq/sub-1110/dtiInit_ses-pre/clean_bundles/dwi_aligned_trilin_space-RASMM_model-DTI_desc-det-AFQ-CGC_L_tractography.t

In [4]:
# after afq api is setup, we can look at plotting

In [5]:
import plotly.graph_objects as go
from plotly.offline import iplot
import numpy as np

In [11]:
fig = go.Figure(data=go.Bar(y=[ord('a'), ord('f'), ord('q')]))
iplot(fig)
# double click to zoom out

In [10]:
x_pts = [10, 12, 15, 18]
y_pts = [5, 1, 1, 2]
z_pts = [10, 10, 8, 7]
# figure = go.Figure()
# figure.add_trace(
#     go.Scatter3d(
#         x=x_pts,
#         y=y_pts,
#         z=z_pts,
#         mode='lines',
#         name='Bundle 1',
#         line=dict(
#             width=1,
#             color='green'
#         )
#     )
# )

In [9]:
x_pts_2 = [8, 9, 14, 16, None, 10, 13, 15, 18]
y_pts_2 = [4, 1, 1, 1, None, 7, 2, 2, 3]
z_pts_2 = [11, 11, 6, 5, None, 10, 11, 8, 6]

# figure.add_trace(
#     go.Scatter3d(
#         x=x_pts_2,
#         y=y_pts_2,
#         z=z_pts_2,
#         mode='lines',
#         name='Bundle 2',
#         line=dict(
#             width=1,
#             color='blue'
#         )
#     )
# )

In [28]:
#myafq.show_inline_bundles() # ~0.5 minutes to run

In [26]:
roi = np.zeros((20, 20, 20))
roi[14, 0:5, 5:11] = 1
roi[13, 1:3, 8:9] = 1
roi[15, 2:4, 7:11] = 1


In [25]:
roi_markers = np.where(roi==1)
figure.add_trace(
    go.Scatter3d(
        x=roi_markers[0],
        y=roi_markers[1],
        z=roi_markers[2],
        name='ROI',
        marker=dict(color='red'),
        line=dict(color=f"rgba(0,0,0,0)")
    )
)

In [24]:
def my_roi(bundle_name):
    figure = myafq.viz.visualize_roi(myafq.export_rois()[0][bundle_name][0], opacity=0.5, name=f'{bundle_name} ROI 1')
    figure = myafq.viz.visualize_roi(myafq.export_rois()[0][bundle_name][1], opacity=0.5, figure=figure, name=f'{bundle_name} ROI 2')
    figure = myafq.viz.visualize_bundles(
        myafq.get_clean_bundles()[0],
        affine=myafq.get_dwi_affine()[0],
        bundle_dict=myafq.bundle_dict,
        bundle=myafq.bundle_dict[bundle_name]['uid'],
        figure=figure, inline=True)
#my_roi('CST_R')

In [56]:
# This will take time
#myafq.show_interactive_ROIs()

INFO:AFQ.viz:Loading Volume...
INFO:AFQ.viz:Loading tractography...
INFO:AFQ.viz:Generating lines from tractography...

KeyboardInterrupt



In [15]:
# Examples of other uses for sliders (ideas?)
display(HTML("""<a href="https://plotly.com/python/sliders/">Slider Examples</a>"""))

In [5]:
# Example of our current use for sliders:
#myafq.viz.visualize_volume(myafq.get_dti_fa()[0], inline=True, which_plane='coronal')

In [16]:
# Examples of other possibly useful plotly graphs
display(HTML("""<a href="https://plotly.com/python/network-graphs/">Network Graphs</a>"""))
display(HTML("""<a href="https://plotly.com/python/v3/continuous-error-bars/">Continuous Error Bars</a>"""))
